In [ ]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,RANSACRegressor,TheilSenRegressor
from statsmodels.robust.robust_linear_model import RLM
import statsmodels
import pandas as pd
from tqdm import tqdm
import random
from scipy import optimize
from sklearn.covariance import MinCovDet as MCD
import time

In [ ]:
main_size = 500

In [ ]:
def Add_Column(df, n = main_size):
    return np.column_stack([np.ones([df.shape[0], 1], dtype=np.int32), df])

In [ ]:
ideal_coefs = np.array([5,2])

In [ ]:
def MC(df, y):
    sum_mnk = 0
    sum_rs = 0
    sum_ts = 0
    sum_m = 0
    sum_mm = 0
    sum_r = 0
    sum_hbr = 0
    sum_lms = 0
    sum_mmr = 0
    sum_hbrr = 0
    nu = 1000
    for i in tqdm(range(nu)):   
        #МНК
        lr = LinearRegression(fit_intercept=False)
        lr.fit(df_norm, y_norm)
        mnk_coefs = lr.coef_
        #ransac
        ransac = RANSACRegressor(lr)
        ransac.fit(df_norm, y_norm)
        ransac_coefs = ransac.estimator_.coef_
        #Theil-Sen Regressor
        ths = TheilSenRegressor(fit_intercept=False)
        ths.fit(df_norm, y_norm)
        ths_coefs = ths.coef_
        #LMS-оценка
        lms = LMS(df_norm, y_norm)
        # M-оценка
        m_est = RLM(y_norm, df_norm, M = statsmodels.robust.norms.TukeyBiweight()).fit()
        m_est_coefs = m_est.params
        ## ММ-оценка
        mm_est = MM(df_norm, y_norm, lms)
        #R-оценка
        beta = optimize.fmin_bfgs(R, 0, args = (df_norm[:,1], y_norm), disp = 0)[0]
        beta_0 = np.median(y_norm - beta*df_norm[:,1])
        r_est = np.array([beta_0, beta])
        #HBR-оценка
        hbr_beta = optimize.fmin_bfgs(HBR, 0, args = (df_norm, y_norm, lms), disp = 0)
        hbr_est = np.array([np.median(y_norm - hbr_beta * df_norm[:,1]),hbr_beta])
        #HBRR-оценка
        hbrr_beta = optimize.fmin_bfgs(HBR, 0, args = (df_norm, y_norm, ransac_coefs), disp = 0)
        hbrr_est = np.array([np.median(y_norm - hbrr_beta * df_norm[:,1]),hbrr_beta])
        #MMR-оценка
        mmr_est = MM(df_norm, y_norm, ransac_coefs)
        #подсчет
        
        sum_mnk += np.linalg.norm(ideal_coefs - mnk_coefs)/nu
        sum_m += np.linalg.norm(ideal_coefs - m_est_coefs)/nu
        sum_ts += np.linalg.norm(ideal_coefs - ths_coefs)/nu
        sum_rs += np.linalg.norm(ideal_coefs - ransac_coefs)/nu
        sum_mm += np.linalg.norm(ideal_coefs - mm_est)/nu
        sum_hbr += np.linalg.norm(ideal_coefs - hbr_est)/nu
        sum_r += np.linalg.norm(ideal_coefs - r_est)/nu
        sum_lms += np.linalg.norm(ideal_coefs - lms)/nu
        sum_mmr += np.linalg.norm(ideal_coefs - mmr_est)/nu
        sum_hbrr += np.linalg.norm(ideal_coefs - hbrr_est)/nu
        
    return sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

In [ ]:
def normal(mean, var, sz = main_size):
    return st.norm.rvs(loc = mean, scale = np.sqrt(var), size = sz)

def unif(a,b):
    return st.uniform.rvs(loc = a, scale = b-a, size = main_size)
    
def tukey(var_1, var_2, bp = 0.1, sz = main_size):
    eps = np.array([])
    for i in range(sz):
        c = st.uniform.rvs()
        if c >= bp:
            eps = np.append(eps, st.norm.rvs(scale = np.sqrt(var_1)))
        else:
            eps = np.append(eps, st.norm.rvs(scale = np.sqrt(var_2)))
    return eps
            
def cauchy(sz = main_size):
    return st.t.rvs(df = 1, size = sz)

In [ ]:
def LMS(df, y):
    h = int(df.shape[0]/2 + 1)
    Q = 1000000000
    beta = np.array([1,1],dtype=float)
    for i in range(df.shape[0]):
        for j in range(i):
            if df[:,1][i] != df[:,1][j]:
                beta[1] = (y[i]-y[j])/(df[:,1][i] - df[:,1][j])
                beta[0] = df[:,1][i]*y[j] - df[:,1][j]*y[i]
                med = np.median(np.abs(y - np.sum(beta*df, axis=1)))
                if med < Q:
                    Q = med.copy()
                    ans = beta.copy()
    return ans

In [ ]:
df_norm = normal(10,5)
eps = normal(0,1)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)
LMS(df_norm, y_norm)

In [ ]:
def MM(df, y, blms):
    n = df.shape[0]
    s = y - np.sum(blms * df, axis = 1)
    res_itog = np.abs(np.median(s) - s)
    MADN = np.median(np.abs(res_itog)) / 0.675
    const1 = 10
    const2 = 4.68
    var = s/(const1*MADN)
    w = np.minimum(3 - 3*np.power(var,2) + np.power(var,4), 1/np.power(var,2))
    sigma_1 = np.dot(w,np.power(s/const1,2))
    sigma_1 = np.sqrt(sigma_1 * 2/ df.shape[0])
    eps = 0.0001
    while(np.abs(sigma_1/MADN - 1) > eps):
        MADN = sigma_1
        sigma_1 = 0
        w = np.minimum(3 - 3*np.power(var,2) + np.power(var,4), 1/np.power(var,2))
        sigma_1 = np.dot(w,np.power(s/const1,2))
        sigma_1 = np.sqrt(sigma_1 * 2 / n)
    var2 = s/(const2*MADN)
    w = np.minimum(3 - 3*np.power(var2,2) + np.power(var2,4), 1/np.power(var2,2))
    W = np.diag(w)
    beta = blms
    beta_1 = np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(df.T,W), df)), df.T), W), y)
    eps = 0.001
    while np.linalg.norm(beta_1-beta)/np.linalg.norm(beta) > eps:
        s = y - np.sum(beta_1 * df, axis = 1)
        var2 = s/(const2*sigma_1)
        w = np.minimum(3 - 3*np.power(var2,2) + np.power(var2,4), 1/np.power(var2,2))
        W = np.diag(w)
        beta = beta_1
        beta_1 = np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(df.T,W), df)), df.T), W), y)
    return beta_1

In [ ]:
def R(x, df, y):
    s = 0
    array = y - x*df
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    return np.dot(array, ranks/(df.shape[0]) - 0.5)

In [ ]:
def HBR(x, df, y, lts):
    resid = y - np.sum(lts * df,axis = 1)
    MADN = 1.483*np.median(np.abs(resid - np.median(resid)))
    mcd = MCD()
    mcd.fit(df[:,1].reshape(-1,1))
    mahal =  mcd.mahalanobis(df[:,1].reshape(-1,1))
    b, c = 4, 4 
    resid = np.abs(resid / MADN)
    resid_new = y - x * df[:,1]
    resid_dif = np.abs(resid_new - resid_new.reshape(-1,1))
    resid_dif = np.tril(resid_dif, -1)
    mahal = b / mahal
    mahal[mahal > 1] = 1
    mahal = mahal / resid
    mahal = mahal.reshape(-1, 1) * mahal
    mahal = np.tril(mahal, -1)
    mahal = c * mahal
    mahal[mahal > 1] = 1
    ans = np.sum(mahal * resid_dif)
    return ans

In [ ]:
start_time = time.time()
lms = LMS(df_norm, y_norm)
hbr_beta = optimize.fmin_bfgs(HBR, 0, args = (df_norm, y_norm, lms), disp = 0)
np.array([np.median(y_norm - hbr_beta * df_norm[:,1]),hbr_beta])
print("--- %s out seconds ---" % (time.time() - start_time))

## $X_1 \sim N(20;2), X_2 \sim N(10;3), \varepsilon \sim N(0;3)$

In [ ]:
norm1 = normal(20,2)
norm2 = normal(10,3)
df_norm = Add_Column(np.concatenate((norm1, norm2)).reshape(2, main_size).transpose())
eps = normal(0,3)
y_norm = 5 + 2*df_norm[:,1] + df_norm[:, 2] + eps

In [ ]:
lr = LinearRegression(fit_intercept=False)
lr.fit(df_norm, y_norm)
mnk_coefs = lr.coef_

In [ ]:
ransac = RANSACRegressor(lr)
ransac.fit(df_norm, y_norm)
ransac_coefs = ransac.estimator_.coef_

In [ ]:
ths = TheilSenRegressor(fit_intercept=False)
ths.fit(df_norm, y_norm)
ths_coefs = ths.coef_

# Нет кластера выбросов

#  $y = 5 + 2X + \varepsilon$

### $X \sim N(20;2), \varepsilon \sim N(0;3)$

In [ ]:
df_norm = normal(20, 2)
eps = normal(0, 3)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts,sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts,sum_mmr, sum_hbrr

In [ ]:
sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim N(20;2), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = normal(20, 2)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

### $X \sim N(10;5), \varepsilon \sim C(0;1)$,

In [ ]:
eps = cauchy()
y_norm = 5 + 2*df_norm[:,1] + eps
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim N(0;3)$

In [ ]:
df_norm = unif(3,7)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim C(0;1)$

In [ ]:
eps = cauchy()
y_norm = 5 + 2*df_norm[:,1] + eps

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5; 200), \varepsilon \sim N(0; 3)$

In [ ]:
df_norm = tukey(5,200)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5;200), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
eps = tukey(1,100)
y_norm = 5 + 2*df_norm[:,1] + eps

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5;200), \varepsilon \sim C(0;1)$

In [ ]:
eps = cauchy()
y_norm = 5 + 2*df_norm[:,1] + eps

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim C(0;1), \varepsilon \sim N(0;3)$

In [ ]:
df_norm = cauchy()
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim C(0;1), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
eps = tukey(1,100)
y_norm = 5 + 2*df_norm[:,1] + eps

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim C(0;1), \varepsilon \sim C(0;1)$

In [ ]:
eps = cauchy()
y_norm = 5 + 2*df_norm[:,1] + eps

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

# Кластер выбросов

### $X \sim N(10;5), \varepsilon \sim N(0;3)$

In [ ]:
df_norm = normal(10,5)
eps = normal(0,3)
a = np.arange(main_size)
random.shuffle(a)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 1000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts,sum_mmr, sum_hbrr

In [ ]:
sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim N(10;5), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = normal(10,5)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 5000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim N(10;5), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = normal(10,5)
eps = cauchy()
a = np.arange(main_size)
random.shuffle(a)
eps[a[:50]] += 5000
y_norm = 5 + 2*df_norm + eps
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim N(0;3)$

In [ ]:
df_norm = unif(3,7)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 10000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = unif(3,7)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 5000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = unif(3,7)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
y_norm[250:] += 5000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5; 200), \varepsilon \sim N(0; 3)$

In [ ]:
df_norm = tukey(5,200)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
y_norm[250:] += 5000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5;200), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = tukey(5,200)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
y_norm[250:] += 10000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5;200), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = tukey(5,200)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
y_norm[250:] += 5000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim C(0;1), \varepsilon \sim N(0;3)$

In [ ]:
df_norm = cauchy()
eps = normal(0,3)
a = np.arange(100)
random.shuffle(a)
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 10000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim C(0;1), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = cauchy()
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
y_norm[:50] += 5000
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

# Засорение иксов

In [ ]:
df_norm = normal(10,5)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts,sum_mmr, sum_hbrr

In [ ]:
sum_mm, sum_hbr, sum_mmr, sum_hbrr

# ОТСЮДА

### $X \sim N(10;5), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = normal(10,5)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim N(10;5), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = normal(10,5)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
df_norm[:30] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim N(0;3)$

In [ ]:
df_norm = unif(3,7)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm[:30] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = unif(3,7)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
df_norm[:30] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim U(3;7), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = unif(3,7)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5; 200), \varepsilon \sim N(0; 3)$

In [ ]:
df_norm = tukey(5,200)
eps = normal(0,3)
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

### $X \sim CN(0.1;5;200), \varepsilon \sim CN(0.1;1;100)$

In [ ]:
df_norm = tukey(5,200)
eps = tukey(1,100)
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr

# Выше моделировалось в ночь!

### $X \sim CN(0.1;5;200), \varepsilon \sim C(0;1)$

In [ ]:
df_norm = tukey(5,200)
eps = cauchy()
y_norm = 5 + 2*df_norm + eps
df_norm[:50] += np.random.uniform(4995, 5000, size = 50)
df_norm = Add_Column(df_norm)

In [ ]:
sum_mnk, sum_m, sum_mm, sum_r, sum_hbr, sum_lms, sum_rs, sum_ts, sum_mmr, sum_hbrr = MC(df_norm, y_norm)

In [ ]:
sum_mnk, sum_m, sum_r, sum_lms, sum_rs, sum_ts, sum_mm, sum_hbr, sum_mmr, sum_hbrr